In [6]:
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy

device = "cuda" if torch.cuda.is_available() else "cpu"

class RBFlayer(nn.Module):
    def __init__(self, timelag):
        super(RBFlayer, self).__init__()

        self.timelag = timelag

        device = "cuda" if torch.cuda.is_available() else "cpu"
        torch.cuda.manual_seed(0)

        self.init_weight_cause = nn.Parameter(torch.rand(self.timelag, device=device))
        self.init_weight_target = nn.Parameter(torch.rand(self.timelag, device=device))
        self.cause_clt = self.init_clt()
        self.cause_std = self.init_clt()
        self.target_clt = nn.Parameter(torch.rand(1, device=device))
        self.target_std = nn.Parameter(torch.rand(1, device=device))

    def init_clt(self):
        return nn.Parameter(torch.rand(1, device=device))

    def init_std(self):
        return nn.Parameter(torch.rand(1, device=device))

    def rbf(self, x, cluster, std):
        return torch.exp(-(x - cluster) * (x - cluster) / 2 * (std * std))

    '''
    def rbf_grad(self, x, cluster, std):
        return (-2 * (x - cluster) / (std * std)) * (torch.exp(-(x - cluster) * (x - cluster) / 2 * (std * std)))
    
    def rbf_grad_num(self, x, x2, cluster, std):
        return (self.rbf(x2,cluster,std) - self.rbf(x, clt, std)) / (x2- x)
    
    def rbf_grad_cause_list(self, X):
        return [self.rbf_grad(X[i+1], self.cause_clt, self.cause_std) for i in range(len(X) - 2)]

    def rbf_grad_target_list(self, X):
        return [self.rbf_grad(X[i+1], self.target_clt, self.target_std) for i in range(len(X) - 2)]

    def rbf_grad_target_num(self, X):
        return
    '''
    def forward(self, cause, target):

        for i in range(len(cause)):
            if i == 0:
                a = self.rbf(cause[i], self.cause_clt, self.cause_std)
            else:
                a = torch.cat([a, self.rbf(cause[i], self.cause_clt, self.cause_std)], dim=0)
        cause = self.init_weight_cause * a

        for j in range(len(target)):
            if j == 0:
                b = self.rbf(target[j], self.target_clt, self.target_std)
            else:
                b = torch.cat([b, self.rbf(target[j], self.target_clt, self.target_std)], dim=0)
        target = self.init_weight_target * b

        return cause, target


class RBFnet(nn.Module):
    def __init__(self, input_size , output_size, timelag):
        super(RBFnet,self).__init__()

        self.input_size = input_size      # number of data
        self.output_size = output_size
        self.timelag = timelag

        self.linear = nn.ModuleList([nn.Linear(self.timelag*2,1) for _ in range(self.input_size)])
        self.relu = nn.ReLU()
        self.networks = nn.ModuleList([RBFlayer(self.timelag) for _ in range(self.input_size)])

    def cause_target(self, cause, target):
        x = torch.cat((cause, target), 0)

        return x

    def GC(self, threshold=True):
        '''
        Extract learned Granger causality.
        Args:
          threshold: return norm of weights, or whether norm is nonzero.
        Returns:
          GC: (p x p) matrix. Entry (i, j) indicates whether variable j is
            Granger causal of variable i.
        '''
        GC = [torch.norm(net.init_weight_cause, dim=0)
              for net in self.networks]
        GC = torch.stack(GC)
        if threshold:
            return (GC > 0).int()
        else:
            return GC


    def forward(self, causes, targets):
        out_list = []
        for i in range(self.input_size):
            cause, target = self.networks[i](causes[i], targets[i])
            cause, target = self.relu(cause), self.relu(target)
            pred = torch.cat((cause, target),0)
            pred = self.linear[i](pred)
            out_list.append(pred)

        return out_list


def restore_parameters(model, best_model):
    '''Move parameter values from best_model to model.'''
    for params, best_params in zip(model.parameters(), best_model.parameters()):
        params.data = best_params

def rbf_p(x, clt, std):
    return (-2 * (x - clt) / (std * std)) * (torch.exp(-(x - clt) * (x - clt) / 2 * (std * std)))

def rbf_fn(x, cluster, std):
    return torch.exp(-(x - cluster) * (x - cluster) / 2*(std * std))


def rbf_grad_cause(model, input):

    rbf_grad_list = []
    for i in range(input.shape[0]):
        list_ = []
        clt = model.networks[i].cause_clt
        std = model.networks[i].cause_std

        for j in range(input.shape[1] - 2):
            list_.append(rbf_p(input[i][j + 1], clt, std))

        rbf_grad_list.append(list_)

    return torch.Tensor(rbf_grad_list)


def rbf_grad_target(model, input):

    rbf_grad_list = []
    for i in range(input.shape[0]):
        list_ = []
        clt = model.networks[i].target_clt
        std = model.networks[i].target_std

        for j in range(input.shape[1] - 2):
            list_.append(rbf_p(input[i][j + 1], clt, std))

        rbf_grad_list.append(list_)

    return torch.Tensor(rbf_grad_list)


def rbf_grad_num_cause(input):
    rbf_grad_list = []
    for j in range(input.shape[0]):
        list_ = []
        clt = model.networks[j].cause_clt
        std = model.networks[j].cause_std

        for i in range(input.shape[1] - 2):
            list_.append((rbf_fn(input[j][i+2],clt,std) - rbf_fn(input[j][i], clt, std)) /
                                        (input[j][i+2] - input[j][i]))

        rbf_grad_list.append(list_)

    return torch.Tensor(rbf_grad_list)

def rbf_grad_num_target(input):
    rbf_grad_list = []
    for j in range(input.shape[0]):
        list_ = []
        clt = model.networks[j].target_clt
        std = model.networks[j].target_std

        for i in range(input.shape[1] - 2):
            list_.append((rbf_fn(input[j][i+2],clt,std) - rbf_fn(input[j][i], clt, std)) /
                                        (input[j][i+2] - input[j][i]))

        rbf_grad_list.append(list_)

    return torch.Tensor(rbf_grad_list)

def train_rbf(model, input_causes, input_targets, Y, lr, epochs, lookback=5,device = device):
    # input_causes, input_targets : X
    # Y : Y
    model.to(device)
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_loss_list = []

    best_it = None
    best_model = None
    best_loss = np.inf

    for epoch in range(epochs):
        pred = model(input_causes, input_targets)
        loss_ = sum([loss_fn(pred[i], Y[i]) for i in range(len(Y))])
        loss_cause = loss_fn(rbf_grad_num_cause(input_causes), rbf_grad_cause(model, input_causes))
        loss_target = loss_fn(rbf_grad_num_target(input_targets), rbf_grad_target(model, input_targets))

        loss = loss_/ len(Y) + loss_cause + loss_target
        print("epoch {} loss {} :".format(epoch, loss / len(Y)))

        loss.backward()
        optimizer.step()
        model.zero_grad()

        train_loss_list.append(loss)
        if loss < best_loss:
            best_loss = loss
            best_it = epoch
            best_model = deepcopy(model)
        elif (epoch - best_it) == lookback:
            if verbose:
                print('Stopping early')
            break

    restore_parameters(model, best_model)

    return train_loss_list , model


In [7]:
def data_split(X, cause, target, timelag, device = device):
    input_cause = []
    input_target = []
    Y = []

    for i in range(len(X) - (timelag + 1)):
        input_cause.append(X[cause].values[i: i + timelag])
        input_target.append(X[target].values[i: i + timelag])
        Y.append([X[target][i + timelag + 1]])

    return torch.tensor(input_cause, device=device).float(), torch.tensor(input_target,device=device).float(), torch.tensor(Y, device=device).float()


In [8]:
import pandas as pd
df = pd.read_csv('C:/Users/chanyoung/Desktop/Neural-GC-master/lorenz_96_10_10_1000.csv')

In [9]:
X2d = df[['a','b']]
torch.manual_seed(1234)
input_cause, input_target, Y = data_split(X2d, 'a', 'b', 10)

In [10]:
model = RBFnet(input_cause.size()[0], 1, input_cause.size()[1])
print(model.networks[1].init_weight_cause)
print(model.networks[1].cause_std)
print(model.networks[1].target_std)
loss, best_model = train_rbf(model, input_cause, input_target, Y, 0.01, 50, device)
print(best_model.networks[1].init_weight_cause)
print(best_model.networks[1].cause_std)
print(model.networks[1].target_std)

Parameter containing:
tensor([0.3990, 0.5167, 0.0249, 0.9401, 0.9459, 0.7967, 0.4150, 0.8203, 0.2290,
        0.9096], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.7874], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.4503], device='cuda:0', requires_grad=True)
epoch 0 loss 0.09041088819503784 :
epoch 1 loss 0.09129206836223602 :
epoch 2 loss 0.09430349618196487 :
epoch 3 loss 0.09948369860649109 :
epoch 4 loss 0.10693389177322388 :
epoch 5 loss 0.11684545129537582 :
epoch 6 loss 0.1295550912618637 :
epoch 7 loss 0.14557567238807678 :
epoch 8 loss 0.16562244296073914 :
epoch 9 loss 0.1906483769416809 :
epoch 10 loss 0.22192206978797913 :
epoch 11 loss 0.26114943623542786 :
epoch 12 loss 0.3106226921081543 :
epoch 13 loss 0.3734202980995178 :
epoch 14 loss 0.4537034332752228 :
epoch 15 loss 0.5571730732917786 :
epoch 16 loss 0.6917473673820496 :
epoch 17 loss 0.8685186505317688 :
epoch 18 loss 1.103216290473938 :
epoch 19 loss 1.418560147285

KeyboardInterrupt: 

In [ ]:
X2d = df[['a','c']]
torch.manual_seed(1234)
input_cause2, input_target2, Y = data_split(X2d, 'a', 'c', 10)
model2 = RBFnet(input_cause.size()[0], 1, input_cause.size()[1])
print(model2.networks[1].init_weight_cause)
print(model2.networks[1].cause_std)
print(model2.networks[1].target_std)
loss2, best_model2 = train_rbf(model2, input_cause2, input_target2, Y, 0.01, 120, device)
print(best_model2.networks[1].init_weight_cause)
print(best_model2.networks[1].cause_std)
print(model2.networks[1].target_std)

Parameter containing:
tensor([0.3990, 0.5167, 0.0249, 0.9401, 0.9459, 0.7967, 0.4150, 0.8203, 0.2290,
        0.9096], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.7874], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.4503], device='cuda:0', requires_grad=True)
epoch 0 loss 25.315834045410156 :
epoch 1 loss 24.86098289489746 :
epoch 2 loss 24.413551330566406 :
epoch 3 loss 23.971946716308594 :
epoch 4 loss 23.534242630004883 :
epoch 5 loss 23.098466873168945 :


In [ ]:
X2d = df[['a','c']]
torch.manual_seed(1234)
input_cause3, input_target3, Y = data_split(X2d, 'a', 'c', 10)
model3 = RBFnet(input_cause.size()[0], 1, input_cause.size()[1])
print(model3.networks[1].init_weight_cause)
print(model3.networks[1].cause_std)
print(model3.networks[1].target_std)
loss3, best_model3 = train_rbf(model3, input_cause3, input_target3, Y, 0.01, 120, device)
print(best_model3.networks[1].init_weight_cause)
print(best_model3.networks[1].cause_std)
print(model3.networks[1].target_std)